# Análise de Inconsistência
Na base de pacientes *classifiacdos com lúpus, de acordo com o algoritmo 2, e com cirugia abdominal*

In [1]:
import pandas as pd
import plotly.express as px

In [2]:
df_lupus_cirurgia = pd.read_parquet('data/results/lupus_cirurgia.parquet')
df_aih = pd.read_parquet(f'data/AIH_Lupus_L93M32N08_todos_cids.parquet')

In [3]:
total_pacientes = df_lupus_cirurgia['id_paciente'].nunique()
print(' - Total de pacientes operados com lúpus:', total_pacientes)

 - Total de pacientes operados com lúpus: 6386


## Sexo

In [4]:
df_count_sexo = df_lupus_cirurgia.groupby('id_paciente')['sexo'].nunique().to_frame()
df_count_sexo.value_counts(normalize=True)*100

sexo
1       99.827748
2        0.172252
Name: proportion, dtype: float64

In [5]:
pacientes_sexo_valido = list(set(df_count_sexo[df_count_sexo['sexo']==1].index))
len(pacientes_sexo_valido)

6375

In [6]:
df_sexo_valido = df_lupus_cirurgia[df_lupus_cirurgia['id_paciente'].isin(pacientes_sexo_valido)]
df_sexo_valido = df_sexo_valido.groupby('sexo')['id_paciente'].nunique().reset_index()
df_sexo_valido.rename(columns={'id_paciente':'Número de Pacientes'}, inplace=True)
df_sexo_valido['Porcentagem'] = df_sexo_valido['Número de Pacientes'] / len(pacientes_sexo_valido)
df_sexo_valido

,sexo,Número de Pacientes,Porcentagem
0,FEMININO,5526,0.866824
1,MASCULINO,849,0.133176


In [7]:
px.bar(df_sexo_valido, x='sexo', y='Porcentagem', text='Número de Pacientes',
       title='Distribuição do Sexo entre os Pacientes')

### Dados Sociodemográficos em cada base
- APAC: idade, sexo, endereço hospital, endereço paciente (uf, município, cep, endereço)
- BPAI: idade*, sexo, endereço hospital, endereço paciente (uf, município, cep (nulo), endereço)
- AIH: idade, sexo, endereço (uf, município) \
- (não está especificado de quem é o endereço, mas como alguns tem poucos registros, deve ser do paciente)
- *BPAI tem idade mas não tem data

### Raça
- Apenas a base AIH possui o campo raça
- Cerca de 80% dos registros da AIH estão preenchidos com "SEM INFORMAÇÂO", mas é possível buscar a informação do paciente nos registros preenchidos
- Cerca de 59% dos pacientes possuem alguma inconsistência no campo raça (o campo varia ao longo dos registros)
- Cerca de 8% dos pacientes não possuem informação sobre raça (contando os que não aparecem na aih ou só aparecem com o tipo 'SEM INFORMAÇÂO')


In [8]:
# Como só a aih tem o campo raça, vale olhar separadamente
# Pacientes com mais de uma raça
df_aih_lupus_cirurgia = df_aih[df_aih['id_paciente'].isin(set(df_lupus_cirurgia['id_paciente']))]
df_outros_lupus_cirurgia = df_lupus_cirurgia[~df_lupus_cirurgia['id_paciente'].isin(set(df_aih['id_paciente'].unique()))]
df_count_raca = (df_aih_lupus_cirurgia.groupby('id_paciente')['raca'].nunique() \
                 .to_frame().value_counts(normalize=True)*100).reset_index()
df_count_raca.rename(columns={'raca':'Número de Raças', 'proportion':'Porcentagem'}, inplace=True)
df_count_raca

,Número de Raças,Porcentagem
0,2,43.778440
1,1,40.828495
2,3,13.996548
3,4,1.333752
4,5,0.062765


In [9]:
aux = df_aih_lupus_cirurgia.groupby('id_paciente')['raca'].nunique()
pacientes_consistentes = aux[aux==1]
print(' - Pacientes com o campo raca consistente:', len(pacientes_consistentes))

 - Pacientes com o campo raca consistente: 2602


In [10]:
px.bar(df_count_raca, x='Número de Raças', y='Porcentagem', color='Porcentagem',
       title='Número de raças registradas para os mesmos pacientes operados da base AIH')

### Distribuição das raças

In [11]:
print(' - Pacientes que aparecem na base AIH:', df_aih_lupus_cirurgia['id_paciente'].nunique())
print(' - Pacientes que não aparecem na base AIH:', df_outros_lupus_cirurgia['id_paciente'].nunique())

 - Pacientes que aparecem na base AIH: 6373
 - Pacientes que não aparecem na base AIH: 13


In [12]:
df_consistentes = df_aih_lupus_cirurgia[df_aih_lupus_cirurgia['id_paciente'].isin(pacientes_consistentes.index)]
df_consistentes['id_paciente'].nunique()
df_consistentes.groupby('raca')['id_paciente'].nunique().sort_values(ascending=False)/df_aih_lupus_cirurgia['id_paciente'].nunique()

raca
BRANCA            0.221246
PARDA             0.090695
SEM INFORMACAO    0.077671
PRETA             0.016162
AMARELA           0.002197
INDIGENA          0.000314
Name: id_paciente, dtype: float64

In [13]:
df_raca = df_consistentes.groupby('raca')['id_paciente'].nunique() \
            .sort_values(ascending=False).to_frame().reset_index()
df_raca.rename(columns={'id_paciente':'Número de Pacientes', 'raca':'Raça'}, inplace=True)

df_raca.loc[2, 'Número de Pacientes'] = df_raca.loc[2, 'Número de Pacientes'] + \
        df_outros_lupus_cirurgia['id_paciente'].nunique() # pacientes não estão na aih estão sem o campo raça

df_raca['Porcentagem'] = df_raca['Número de Pacientes']*100/ total_pacientes
df_raca.sort_values(by='Porcentagem', ascending=False, inplace=True)
px.bar(df_raca, x='Raça', y='Porcentagem', color='Raça', text='Número de Pacientes',
       title='Distribuição das Raças entre os Pacientes')

## Outros Campos
Os únicos campo utilizados até aqui que apresentam algum tipo de valores faltantes são 'raça' e 'data'.

No caso da raça, este campo aparece apenas na base AIH. E no caso do campo 'data', ele não existe na base BPAI.

In [14]:
print(' - Pacientes sem o campo raça:', df_outros_lupus_cirurgia['id_paciente'].nunique())

 - Pacientes sem o campo raça: 13


In [15]:
df_lupus_cirurgia['data_invalida'] = df_lupus_cirurgia['data'].dt.year==2058
porcent_data_faltante = float(df_lupus_cirurgia['data_invalida'].sum()*100/df_lupus_cirurgia.shape[0])
print(' - Porcentagem de pacientes com a data do diagnóstico de lúpus faltando:', round(porcent_data_faltante,2),'%')

 - Porcentagem de pacientes com a data do diagnóstico de lúpus faltando: 0.23 %


In [16]:
print(' - Mostrando que não tem existem valores nulos na base dos pacientes operados')
df_lupus_cirurgia.isna().sum()

 - Mostrando que não tem existem valores nulos na base dos pacientes operados


id_paciente                  0
sexo                         0
idade                        0
uf_paciente                  0
data                         0
co_procedimento_principal    0
regiao                       0
idade_categoria              0
data_invalida                0
dtype: int64

### Base AIH

In [17]:
df_aux = pd.DataFrame.from_dict({'Coluna': ['co_cid_obito', 'raca'],
        'Porcentagem de Nulos': [df_aih[df_aih['co_cid_obito'] == '    '].shape[0]*100/len(df_aih),
                                df_aih[df_aih['raca'] == 'SEM INFORMACAO'].shape[0]*100/df_aih.shape[0]] })
df_aux

,Coluna,Porcentagem de Nulos
0,co_cid_obito,51.738068
1,raca,22.604288


In [18]:
print(' -  Colunas na AIH com porcantagem de nulos maior que zero:')
df_aih_na = df_aih.isna().sum()
df_aih_na = (df_aih_na[df_aih_na>0]*100 / df_aih.shape[0]).reset_index()
df_aih_na.rename(columns={'index':'Coluna', 0:'Porcentagem de Nulos'}, inplace=True)
df_aih_na = pd.concat([df_aih_na, df_aux])
df_aih_na.sort_values('Porcentagem de Nulos', ascending=False).apply(lambda row: round(row, 2))

 -  Colunas na AIH com porcantagem de nulos maior que zero:


,Coluna,Porcentagem de Nulos
1,co_cid_secundario_1,54.90
2,co_cid_secundario_2,54.90
3,co_cid_secundario_3,54.90
4,co_cid_secundario_4,54.90
8,co_cid_secundario_8,54.90
5,co_cid_secundario_5,54.90
6,co_cid_secundario_6,54.90
7,co_cid_secundario_7,54.90
9,co_cid_secundario_9,54.90
0,co_cid_obito,51.74


In [19]:
df_aih['raca'].value_counts()

raca
BRANCA            1557842
PARDA             1208630
SEM INFORMACAO     897152
PRETA              253704
AMARELA             48274
INDIGENA             3344
Name: count, dtype: int64

### BPAI

In [20]:
df_bpai = pd.read_parquet(f'data/BPAI_Lupus_L93M32N08_todos_cids.parquet')

In [21]:
print(' - Colunas com valores nulos no BPAI:')
df_bpai_na = df_bpai.isna().sum().sort_values(ascending=False)*100/len(df_bpai)
df_bpai_na[df_bpai_na>0].apply(lambda row: round(row, 2))

 - Colunas com valores nulos no BPAI:


cns_paciente    100.00
no_paciente     100.00
co_municipio     11.22
valor             0.00
dtype: float64

### APAC

In [22]:
df_apac = pd.read_parquet(f'data/APAC_Lupus_L93M32N08_todos_cids.parquet')

In [23]:
print(' - Colunas com valores nulos no BPAI:')
df_apac_na = df_apac.isna().sum().sort_values(ascending=False)*100/len(df_apac)
df_apac_na[df_apac_na>0].apply(lambda row: round(row, 2))

 - Colunas com valores nulos no BPAI:


cpf_diretor              100.00
cns_diretor              100.00
responsavel              100.00
mae                      100.00
endereco                 100.00
nome_pasc                100.00
cns_pasc                 100.00
num_endereco             100.00
cep                      100.00
complemento_endereco     100.00
cns_solicitante          100.00
cpf_solicitante          100.00
cns_medico                99.21
co_rubrica                44.44
co_classificacao          41.28
co_inst_registro          41.28
co_servico                41.28
co_municipio_hospital      2.72
dtype: float64

## Verificando Sexo e Próstota

In [24]:
def porcentagem_de_prostata(df, col_proc_principal, col_proc_secundario=None):        
    if col_proc_secundario is not None:
        df['PROSTATA_proced_principal'] = df[col_proc_principal].apply(lambda row: 'PROST' in row)
        df['PROSTATA_proced_secundario'] = df[col_proc_secundario].apply(lambda row: 'PROST' in row)
        df['PROSTATA'] = df['PROSTATA_proced_principal'] | df['PROSTATA_proced_secundario']
    else:
        df['PROSTATA'] = df[col_proc_principal].apply(lambda row: 'PROST' in row)
    print(' - Porcentagem de pacientes com procedimentos relacadionados à prostata:',\
        round(float(df['PROSTATA'].sum()*100/len(df)),3))
    
def veririca_inconsistencia(df, col_sexo, val_sexo):
    df_incosistencia_prostata = df.loc[(df[col_sexo]==val_sexo) & (df['PROSTATA'])]
    print(' - Total de registros com incosistência:', df_incosistencia_prostata.shape[0])
    print(' - Total de pacientes com incosistência:', df_incosistencia_prostata['id_paciente'].nunique())

    if(df_incosistencia_prostata['id_paciente'].nunique() > 0):
        df_ipc = df_incosistencia_prostata['id_paciente'].isin(df_lupus_cirurgia['id_paciente'].unique())
        # Dentre os classificados com lúpus e cirurgia abdominal:
        print(' - Dentre os classificados com lúpus e cirurgia abdominal:', df_incosistencia_prostata[df_ipc]['id_paciente'].nunique())
        print(' - São eles:', df_incosistencia_prostata[df_ipc]['id_paciente'].unique())


### BPAI

In [25]:
porcentagem_de_prostata(df_bpai, 'no_procedimento_realizado')
veririca_inconsistencia(df_bpai, 'sexo', 'F')

 - Porcentagem de pacientes com procedimentos relacadionados à prostata: 0.038
 - Total de registros com incosistência: 0
 - Total de pacientes com incosistência: 0


In [26]:
print(' -  Procedimentos relacionados à prostata:')
df_bpai[df_bpai['PROSTATA']][['no_procedimento_realizado']].value_counts()

 -  Procedimentos relacionados à prostata:


no_procedimento_realizado                      
DOSAGEM DE ANTIGENO PROSTATICO ESPECIFICO (PSA)    148
ULTRASSONOGRAFIA DE PROSTATA (VIA TRANSRETAL)      102
ULTRASSONOGRAFIA DE PROSTATA POR VIA ABDOMINAL      69
BIOPSIA DE PROSTATA                                  4
Name: count, dtype: int64

### Na AIH

In [27]:
porcentagem_de_prostata(df_aih, 'procedimento_principal', 'desc_procedimento_secundario')
veririca_inconsistencia(df_aih, 'co_paciente_sexo', 'F')

 - Porcentagem de pacientes com procedimentos relacadionados à prostata: 0.061
 - Total de registros com incosistência: 14
 - Total de pacientes com incosistência: 13
 - Dentre os classificados com lúpus e cirurgia abdominal: 2
 - São eles: [16761197800  1479600700]


In [28]:
df_aih_prostata = df_aih[df_aih['PROSTATA']][['procedimento_principal']]
print(' -  Procedimentos relacionados à prostata:', df_aih_prostata.nunique())
print()
print(' - Os 20 mais frequentes:')
df_aih[df_aih['PROSTATA']][['procedimento_principal']].value_counts().iloc[:20]

 -  Procedimentos relacionados à prostata: procedimento_principal    84
dtype: int64

 - Os 20 mais frequentes:


procedimento_principal                                                                    
RESSECCAO ENDOSCOPICA DE PROSTATA                                                             744
PROSTATECTOMIA SUPRAPÚBICA                                                                    459
PROSTATOVESICULECTOMIA RADICAL EM ONCOLOGIA                                                   305
PROSTATECTOMIA EM ONCOLOGIA                                                                   201
TRATAMENTO DE INSUFICIENCIA RENAL CRONICA                                                      98
TRATAMENTO DE DOENCAS GLOMERULARES                                                             88
PROSTATOVESICULECTOMIA RADICAL                                                                 72
TRATAMENTO C/ CIRURGIAS MULTIPLAS                                                              60
TRATAMENTO DE INTERCORRÊNCIA PÓS-TRANSPLANTE DE ÓRGÃOS / CÉLULAS-TRONCO HEMATOPOÉTICAS         39
PROCEDIMENTOS SEQUENCIAIS E

### APAC

In [29]:
porcentagem_de_prostata(df_apac, 'no_procedimento_principal', 'no_procedimento_secundario')
veririca_inconsistencia(df_apac, 'sexo', 'FEMININO')

 - Porcentagem de pacientes com procedimentos relacadionados à prostata: 0.037
 - Total de registros com incosistência: 2054
 - Total de pacientes com incosistência: 72
 - Dentre os classificados com lúpus e cirurgia abdominal: 9
 - São eles: [ 1011194700 17870977200 11435917500 16069144600   241742700 19870618700
 13550198400 22643198700  9309814200]


In [30]:
print(' -  Procedimentos relacionados à prostata:')
df_apac[df_apac['PROSTATA']][['no_procedimento_principal']].value_counts()

 -  Procedimentos relacionados à prostata:


no_procedimento_principal                                       
BIMATOPROSTA 0,3 MG/ML SOLUÇÃO OFTÁLMICA (POR FRASCO DE 3 ML)       1294
LATANOPROSTA 0,05 MG/ML SOLUÇÃO OFTÁLMICA (POR FRASCO DE 2,5 ML)     710
TRAVOPROSTA 0,04 MG/ML SOLUÇÃO OFTÁLMICA (POR FRASCO DE 2,5 ML)      667
ILOPROSTA 10 MCG/ML SOLUÇÃO PARA NEBULIZAÇÃO (AMPOLA DE 1 ML)          1
Name: count, dtype: int64